In [ ]:
# import os
import torch
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from torchinfo import summary
from torch.utils.data import DataLoader

from models.deepLOB.deepLOB_model import Deeplob
from models.transLOB import TransLOB
from utils.fi2010_loader import Dataset_fi2010
from utils.training import train_validate
from utils.evaluation import evaluate
from utils.plots import plot_training_history, plot_label_distributions

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
dataset_type='fi2010'
normalization = 'Zscore'
lighten = False # True
feature_size = 40

T = 100
k = 4
stock = [0, 1, 2, 3, 4]
train_test_ratio = 0.75
days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

batch_size= 128
num_workers= 4

In [ ]:
train_day_length = round(len(days) * train_test_ratio)
train_days = days[:train_day_length]
test_days = days[train_day_length:]

dataset_train = Dataset_fi2010(auction, normalization, stock, train_days, T, k, lighten, "deeplob")
dataset_val = Dataset_fi2010(auction, normalization, stock, test_days, T, k, lighten, "deeplob")

print(f"Training Data Size : {dataset_train.__len__()}")
print(f"Validation Data Size : {dataset_val.__len__()}")

train_loader = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
plot_label_distributions(dataset_train, dataset_val)

# DeepLOB

In [ ]:
model_deepLOB = Deeplob(lighten=lighten).to(device)
summary(model, (1, 1, 100, feature_size))

In [ ]:
epochs = 10
learning_rate = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_deepLOB.parameters(), lr=learning_rate)

history = train_validate(model_deepLOB, train_loader, val_loader, optimizer, criterion, epochs, k, dataset_type, device)

In [ ]:
plot_training_history(history)

In [ ]:
evaluate(model_deepLOB, val_loader, device)

In [ ]:
# from utils.fi2010_loader import __vis_sample_lob__
# __vis_sample_lob__('DecPre')

# TransLOB

In [ ]:
model_transLOB = TransLOB(
    num_features=feature_size,
    num_classes=3,
    hidden_channels=14,
    d_model=64,
    num_heads=4,
    num_transformer_blocks=2
).to(device)
summary(model_transLOB, (1, 100, feature_size))

In [ ]:
epochs = 10
learning_rate= 1e-4
ADAM_B1 = 0.9
ADAM_B2 = 0.999
WEIGHT_DECAY= 1e-5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_transLOB.parameters(), lr=learning_rate, betas=(ADAM_B1, ADAM_B2), weight_decay=WEIGHT_DECAY)

history = train_validate(model_transLOB, train_loader, val_loader, optimizer, criterion, epochs, k, dataset_type, device)

In [ ]:
plot_training_history(history)

In [ ]:
evaluate(model_transLOB, val_loader, device)

In [ ]:
# from tqdm import tqdm
# from datetime import datetime

# def batch_gd(model_id, model, criterion, optimizer, train_loader, val_loader, epochs, name):
#     training_info = {
#         'train_loss_hist': [],
#         'val_loss_hist': [],
#         'train_acc_hist': [],
#         'val_acc_hist': []
#     }

#     best_test_loss = np.inf
#     best_test_epoch = 0
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
#     for iter in tqdm(range(epochs)):
#         model.train()
#         t0 = datetime.now()
#         train_loss = []
#         train_acc = []
#         for inputs, targets in tqdm(train_loader):
#             # move data to GPU
#             inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
#             # zero the parameter gradients
#             optimizer.zero_grad()
#             # Forward pass
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             # Backward and optimize
#             loss.backward()
#             optimizer.step()
#             train_loss.append(loss.item())
#             tmp_acc = torch.count_nonzero(torch.argmax(outputs, dim = 1) == targets).item()/targets.size(0)
#             train_acc.append(tmp_acc)
#         # Get train loss and test loss
#         train_loss = np.mean(train_loss)
#         train_acc = np.mean(train_acc)

#         model.eval()
#         val_loss = []
#         val_acc = []
#         for inputs, targets in tqdm(val_loader):
#             inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             val_loss.append(loss.item())
#             tmp_acc = torch.count_nonzero(torch.argmax(outputs, dim=1) == targets).item() / targets.size(0)
#             val_acc.append(tmp_acc)
#         val_loss = np.mean(val_loss)
#         val_acc = np.mean(val_acc)

#         # Save losses
#         training_info['train_loss_hist'].append(train_loss)
#         training_info['val_loss_hist'].append(val_loss)
#         training_info['train_acc_hist'].append(train_acc)
#         training_info['val_acc_hist'].append(val_acc)

#         if val_loss < best_test_loss:
#             # torch.save(model, os.path.join(logger.find_save_path(model_id), 'best_val_model.pt'))
#             best_test_loss = val_loss
#             best_test_epoch = iter
#             print('model saved')

#         dt = datetime.now() - t0
#         print(f'Epoch {iter + 1}/{epochs}, '
#               f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc: .4f}, '
#               f'Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc: .4f}, '
#               f'Duration: {dt}, Best Val Epoch: {best_test_epoch}')

#     # torch.save({
#     #     'epoch': epochs,
#     #     'model_state_dict': model.state_dict(),
#     #     'optimizer_state_dict': optimizer.state_dict(),
#     #     'loss': train_loss,
#     # }, os.path.join(logger.find_save_path(model_id), 'checkpoint.pt'))

#     # with open(os.path.join(logger.find_save_path(model_id), 'training_process.pkl'), 'wb') as f:
#     #     pickle.dump(training_info, f)

#     return


In [ ]:
# model_transLOB.eval()

# all_y_true = []
# all_y_pred = []

# with torch.no_grad():
#     for X_batch, y_batch in val_loader:
#         X_batch, y_batch = X_batch.to(device, dtype=torch.float), y_batch.to(device, dtype=torch.int64) 
#         # X_batch, y_batch = X_batch.to(device), y_batch.to(device)

#         outputs = model_transLOB(X_batch)
#         _, y_pred = outputs.max(1)

#         all_y_true.append(y_batch.cpu().numpy())
#         all_y_pred.append(y_pred.cpu().numpy())

# all_y_true = np.concatenate(all_y_true)
# all_y_pred = np.concatenate(all_y_pred)

# cm = confusion_matrix(all_y_true, all_y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(cmap="Blues", values_format="d")
# plt.title("Confusion Matrix on Test Set")
# plt.show()

# accuracy = accuracy_score(all_y_true, all_y_pred)
# precision = precision_score(all_y_true, all_y_pred, average='macro')
# recall = recall_score(all_y_true, all_y_pred, average='macro')
# f1 = f1_score(all_y_true, all_y_pred, average='macro')

# print(classification_report(all_y_true, all_y_pred, digits=4))

# print(f"Test Accuracy: {accuracy:.4f}")
# print(f"Test Precision (macro): {precision:.4f}")
# print(f"Test Recall (macro): {recall:.4f}")
# print(f"Test F1 Score (macro): {f1:.4f}")

In [ ]:
# from utils.training import train, validate, train_validate




# train_losses = []
# train_accuracies = []
# val_losses = []
# val_accuracies = []

# best_val_acc = 0

# for epoch in range(epochs):
#     train_loss, train_acc = train(model_transLOB, train_loader, optimizer, criterion, device)
#     val_loss, val_acc = validate(model_transLOB, val_loader, criterion, device)
#     # val_loss, val_acc = validate(model, val_loader, criterion, device)

#     train_losses.append(train_loss)
#     train_accuracies.append(train_acc)
#     val_losses.append(val_loss)
#     val_accuracies.append(val_acc)

#     print(f"Epoch {epoch+1}/{epochs}: "
#           f"Train Loss {train_loss:.4f} | Train Acc {train_acc:.4f} | "
#           f"Val Loss {val_loss:.4f} | Val Acc {val_acc:.4f}")

#     # Save best model
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save(model_transLOB.state_dict(), './outputs/transLOB/best_transLOB_fi2010.pth')
#         print(f"✅ Saved best model at epoch {epoch+1} with Val Acc {val_acc:.4f}")